In [1]:
#!pip install pycocotools

In [2]:
import glob
import json
import os
from tqdm import tqdm

from pycocotools.coco import COCO  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb

from utils import imread, imwrite
from vis_prior_generator import CannyVPG, HEDVPG, MLSDVPG, MidasVPG, UniformerVPG
from vis_prior_layout import UniformRandomNoClipVPL

In [3]:
source_image_folder = "/home/ubuntu/dad/roboflow-100-benchmark/rf100/apples-fvpl5/train/"
source_annotation_file = "/home/ubuntu/dad/roboflow-100-benchmark/rf100/apples-fvpl5/train/_annotations.coco.json"
ext = "jpg"
im_prior_folder = "/home/ubuntu/dad/roboflow-100-benchmark/rf100/apples-fvpl5/train_bboxhed/"
annotation_path = "/home/ubuntu/dad/roboflow-100-benchmark/rf100/apples-fvpl5/train_aug_bboxhed.json"  # annotation for controlnet
if not os.path.exists(im_prior_folder):
    os.mkdir(im_prior_folder)

coco = COCO(source_annotation_file)
naive_vpd = UniformRandomNoClipVPL()
vpg = HEDVPG(vpl=naive_vpd, fill_val=0, annotation=None, im_folder=None)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
coco.loadAnns(coco.getAnnIds(imgIds=1))

[{'id': 1,
  'image_id': 1,
  'category_id': 2,
  'bbox': [121, 82, 242, 328.5],
  'area': 79497,
  'segmentation': [],
  'iscrowd': 0}]

In [5]:
img_ids = coco.getImgIds()

with open(annotation_path, "w+") as f:
    for img_id in tqdm(img_ids):
        coco_img = coco.loadImgs(img_id)[0]
        source_img_name = coco_img['file_name']
        
        source_img_path = os.path.join(source_image_folder, source_img_name)
        im_prior_path = os.path.join(im_prior_folder, source_img_name)
        
        source_img = imread(source_img_path)
        annIds = coco.getAnnIds(imgIds=coco_img['id'])
        anns = coco.loadAnns(annIds)
        bboxes = [ann['bbox'] for ann in anns]
        
        im_prior = vpg.detect_one_img(img=source_img,bboxes=bboxes)
        imwrite(im_prior_path, im_prior)
        
        json_row = json.dumps({"source":im_prior_path,"target":source_img_path,"prompt": "apples"})
        f.write(json_row + '\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 489/489 [00:24<00:00, 19.93it/s]


In [6]:
# view on VS Code

In [7]:
annotation_path

'/home/ubuntu/dad/roboflow-100-benchmark/rf100/apples-fvpl5/train_aug_bboxhed.json'